# T1016 - System Network Configuration Discovery
Adversaries may look for details about the network configuration and settings, such as IP and/or MAC addresses, of systems they access or through information discovery of remote systems. Several operating system administration utilities exist that can be used to gather this information. Examples include [Arp](https://attack.mitre.org/software/S0099), [ipconfig](https://attack.mitre.org/software/S0100)/[ifconfig](https://attack.mitre.org/software/S0101), [nbtstat](https://attack.mitre.org/software/S0102), and [route](https://attack.mitre.org/software/S0103).

Adversaries may also leverage a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) on network devices to gather information about configurations and settings, such as IP addresses of configured interfaces and static/dynamic routes (e.g. <code>show ip route</code>, <code>show ip interface</code>).(Citation: US-CERT-TA18-106A)(Citation: Mandiant APT41 Global Intrusion )

Adversaries may use the information from [System Network Configuration Discovery](https://attack.mitre.org/techniques/T1016) during automated discovery to shape follow-on behaviors, including determining certain access within the target network and what actions to do next. 

## Atomic Tests

### Atomic Test #1 - System Network Configuration Discovery on WindowsIdentify network configuration information

Upon successful execution, cmd.exe will spawn multiple commands to list network configuration settings. Output will be via stdout.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
ipconfig /all
netsh interface show interface
arp -a
nbtstat -n
net config
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 1

### Atomic Test #2 - List Windows Firewall RulesEnumerates Windows Firewall Rules using netsh.

Upon successful execution, cmd.exe will spawn netsh.exe to list firewall rules. Output will be via stdout.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh advfirewall firewall show rule name=all
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 2

### Atomic Test #3 - System Network Configuration Discovery
Identify network configuration information.

Upon successful execution, sh will spawn multiple commands and output will be via stdout.

**Supported Platforms:** macos, linux
#### Dependencies:  Run with `sh`!
##### Description: Check if arp command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v arp)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
(which yum && yum -y install net-tools)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y net-tools)

```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
if [ -x "$(command -v arp)" ]; then arp -a; else echo "arp is missing from the machine. skipping..."; fi;
if [ -x "$(command -v ifconfig)" ]; then ifconfig; else echo "ifconfig is missing from the machine. skipping..."; fi;
if [ -x "$(command -v ip)" ]; then ip addr; else echo "ip is missing from the machine. skipping..."; fi;
if [ -x "$(command -v netstat)" ]; then netstat -ant | awk '{print $NF}' | grep -v '[a-z]' | sort | uniq -c; else echo "netstat is missing from the machine. skipping..."; fi;
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 3

### Atomic Test #4 - System Network Configuration Discovery (TrickBot Style)Identify network configuration information as seen by Trickbot and described here https://www.sneakymonkey.net/2019/10/29/trickbot-analysis-part-ii/

Upon successful execution, cmd.exe will spawn `ipconfig /all`, `net config workstation`, `net view /all /domain`, `nltest /domain_trusts`. Output will be via stdout.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
ipconfig /all
net config workstation
net view /all /domain
nltest /domain_trusts
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 4

### Atomic Test #5 - List Open Egress Ports
This is to test for what ports are open outbound.  The technique used was taken from the following blog:
https://www.blackhillsinfosec.com/poking-holes-in-the-firewall-egress-testing-with-allports-exposed/

Upon successful execution, powershell will read top-128.txt (ports) and contact each port to confirm if open or not. Output will be to Desktop\open-ports.txt.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Test requires #{port_file} to exist

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1016\src\top-128.txt") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1016\src\top-128.txt) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1016/src/top-128.txt" -OutFile "PathToAtomicsFolder\T1016\src\top-128.txt"

```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$ports = Get-content PathToAtomicsFolder\T1016\src\top-128.txt
$file = "$env:USERPROFILE\Desktop\open-ports.txt"
$totalopen = 0
$totalports = 0
New-Item $file -Force
foreach ($port in $ports) {
    $test = new-object system.Net.Sockets.TcpClient
    $wait = $test.beginConnect("allports.exposed", $port, $null, $null)
    $wait.asyncwaithandle.waitone(250, $false) | Out-Null
    $totalports++ | Out-Null
    if ($test.Connected) {
        $result = "$port open" 
        Write-Host -ForegroundColor Green $result
        $result | Out-File -Encoding ASCII -append $file
        $totalopen++ | Out-Null
    }
    else {
        $result = "$port closed" 
        Write-Host -ForegroundColor Red $result
        $totalclosed++ | Out-Null
        $result | Out-File -Encoding ASCII -append $file
    }
}
$results = "There were a total of $totalopen open ports out of $totalports ports tested."
$results | Out-File -Encoding ASCII -append $file
Write-Host $results
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 5

#### Cleanup: 
```powershell
Remove-Item -ErrorAction ignore "$env:USERPROFILE\Desktop\open-ports.txt"
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Adfind - Enumerate Active Directory Subnet Objects
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory Subnet Objects
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (#{adfind_path})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1087.002\src\AdFind.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/src/AdFind.exe" -OutFile PathToAtomicsFolder\T1087.002\src\AdFind.exe

```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1087.002\src\AdFind.exe -f (objectcategory=subnet)
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 6

### Atomic Test #7 - Qakbot Recon
A list of commands known to be performed by Qakbot for recon purposes
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: File to copy must exist on disk at specified location (#{recon_commands})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1016\src\qakbot.bat) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path PathToAtomicsFolder\T1016\src\qakbot.bat) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1016/src/qakbot.bat" -OutFile "PathToAtomicsFolder\T1016\src\qakbot.bat"

```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1016\src\qakbot.bat
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 7

### Atomic Test #8 - List macOS Firewall Rules"This will test if the macOS firewall is enabled and/or show what rules are configured. Must be run with elevated privileges. Upon successful execution, these commands will output various information about the firewall configuration, including status and specific port/protocol blocks or allows. 

Using `defaults`, additional arguments can be added to see filtered details, such as `globalstate` for global configuration (\"Is it on or off?\"), `firewall` for common application allow rules, and `explicitauths` for specific rules configured by the user. 

Using `socketfilterfw`, flags such as --getglobalstate or --listapps can be used for similar filtering. At least one flag is required to send parseable output to standard out. 
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
sudo defaults read /Library/Preferences/com.apple.alf
sudo /usr/libexec/ApplicationFirewall/socketfilterfw --getglobalstate
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 8

### Atomic Test #9 - DNS Server Discovery Using nslookupIdentify System domain dns controller on an endpoint using nslookup ldap query. This tool is being abused by qakbot malware to gather information on the domain
controller of the targeted or compromised host. reference https://securelist.com/qakbot-technical-analysis/103931/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
nslookup -querytype=ALL -timeout=12 _ldap._tcp.dc._msdcs.$env:USERDNSDOMAIN
```

In [ ]:
Invoke-AtomicTest T1016 -TestNumbers 9

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Further, {{LinkById|T1059.008} commands may also be used to gather system and network information with built-in features native to the network device platform.  Monitor CLI activity for unexpected or unauthorized use  commands being run by non-standard users from non-standard locations.  Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).


## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
There is an opportunity to influence an adversary to move toward systems you want them to engage with.
#### Use Case
A defender can create breadcrumbs or honeytokens to lure the attackers toward the decoy systems or network services.
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.
